# Use CPlantBox with one click
In this tutorial, you only need to click the small triangle button on the left top corner of the code blocks. 
If the circling triangle finished, please go to the next block and run the next block. 
If you need to learn google colab please follow the link here https://colab.research.google.com/notebooks/welcome.ipynb


In [ ]:
# #<---Click this triangle to load python libraries
# # loading other python library or packages. Only need to run once at the start.
import os
import sys
# ! pip3 install vtk
# from vtk.util import numpy_support as VN
import vtk
! git clone https://github.com/Plant-Root-Soil-Interactions-Modelling/CPlantBox # downloading the source code
os.chdir("tutorial/jupyter") # Change to the python directory (working directory)
# # Congrats! Now you are ready to use the CPlantBox
# !rm modelparameter
# !ln -s /content/CPlantBox/modelparameter/ modelparameter
# Loading specific python scripts for CPlantBox and CRootBox
from CPlantBox_PiafMunch import *
plotly.__version__
# first we create a Heliantus plant
name = "Heliantus_Pagès_2013_new.xml" # parameter name
# here are some optional parameters to be tested
# name = "PMA2018" # Simulate a small plant with 3 leaves and two lateral root, you can comment the heliantus line and uncomment this line to see what happend.
time = 70 # how many days the plant need to grow, make it smaller, for example 15 to see if the plant becomes smaller
plant1 = CPlantBox(name, time, name) # make a plant object in python
# Visualization
fig = visual_plant(plant1)
fig.show()

## 2. Detailed usage of CPlantBox
1. create several different plant.

In [ ]:
# it is easy to use for loop in python to create many plant at a time.
for i in range(0,9):    
    plant1 = CPlantBox_analysis('{}_new.xml'.format(str(i)), 180, '{}'.format(str(i)))

## 3. Rewirte the visualization function


In [ ]:
def visual_plant_sub(plant1,name='plant'):
    nodes_cor = python_nodes(plant1) # use the object name created previously to get its coordinates
    fig= go.Scatter3d(
        x=nodes_cor.T[3],
        y=nodes_cor.T[4],
        z=nodes_cor.T[5],
        mode='markers',
        marker=dict(
            size=3,
            color=nodes_cor.T[1],                # nodes_cor.T[1] is organ type, nodes_cor.T[2] is the connection number of a node 
            colorscale=[[0, "wheat"], #color of the root, change it to "yellow" to see the difference
                    [0.5, "darkgreen"],
                    [1.0, "lightgreen"],],  opacity=0.8
        ), name =name
    )

    return fig

def visual_plant(plant1):
    subfig = visual_plant_sub(plant1)
    fig = make_subplots(
    rows=1, cols=1,
    specs=[[{'type': 'surface'}]])
    fig.add_trace(subfig) 
    fig.update_layout(scene_aspectmode='data',)
    return fig

### 3.2 Visualization through Matplotlib + VTK




In [ ]:
import vtk.util.numpy_support as VN
# CPlantBox("monopodial_new.xml",120, "monopodial")
# CPlantBox("sympodial_dichasium_new.xml",180, "sympodial_dichasium")
# CPlantBox("sympodial_monochasium_new.xml",110, "sympodial_monochasium")
# CPlantBox("leaf_opposite_decussate_new.xml",180, "leaf_opposite_decussate")
# CPlantBox("leaf_alternate_new.xml",180, "leaf_alternate")
# CPlantBox("leaf_spiral_new.xml",180, "leaf_spiral")
h_font = {'fontname':'DejaVu Sans Mono'}
def plot_vtp(name, indentx, indentz,scale , title, oren):
    reader = vtk.vtkXMLPolyDataReader()
    path = "{}.vtp".format(name) #path or name of the vtp output
    reader.SetFileName(path)
    reader.Update()
    polydata = reader.GetOutput()
    vtk_organtype = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("organType"))
    vtk_id = VN.vtk_to_numpy(reader.GetOutput().GetCellData().GetArray("id"))
    #vtk_time = VN.vtk_to_numpy(reader.GetOutput().GetPointData().GetArray("time"))
    vtk_cor = VN.vtk_to_numpy(reader.GetOutput().GetPoints().GetData())
    vtk_line = VN.vtk_to_numpy(reader.GetOutput().GetLines().GetData())
# print(vtk_organtype,"\n",vtk_id)
    ax.text2D(indentx/13 +0.3, indentz/23+0.57, title,rotation=0 , transform=ax.transAxes, size=20,**h_font)
#     ax.set_xlim3d(-0.2,0.2)
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()
        
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        x = [ (n)*scale+indentx-1 for n in np_pts[:,0]/100]
        
        z = [ (n)*scale+indentz for n in np_pts[:,2]/100]

        y = [ n*scale for n in np_pts[:,1]/100]
        ax.plot3D(x,y,z, c= get_organ_color(vtk_organtype[i]) ,linewidth=2)

def get_organ_color(i):
    if i == 2:
        return "wheat"
    elif i == 4:
        return "#5b965b"
    elif i == 8:
        return "lightgreen"
    else:
        return "black";
   

#name = "sympodial_dichasium"
plt.rcParams['figure.figsize'] = [20, 40]
fig = plt.figure("Stem Branching and Leaf Arrangements")
fig.patch.set_alpha(0.)
fig.subplots_adjust(hspace=0, wspace=0)
ax = fig.add_subplot(int("11{}".format(1)), projection='3d')
#plt.axis('image')

plt.title("")
plt.axis('off')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.view_init(45, 270)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_zlabel('')

#ax.set_ylim3d(0,2)
#ax.set_zlim3d(-1.5,0.5)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_zticklabels([])
ax.grid(False)
plot_vtp("monopodial",0,0,1,"Monopodial",1)

plot_vtp("sympodial_dichasium",2,0,1," Sympodial \n dichasium",1)
plot_vtp("sympodial_monochasium",4,0,1," Sympodial \n monochasium",0.5)


plot_vtp("leaf_opposite_decussate",0,-5,50, " Opposite\n decussate",1)
plot_vtp("leaf_alternate",2,-5,50, "Alternate",1)
plot_vtp("leaf_spiral",4,-5,50, "Spiral" ,-1)
ax.text2D(0/13 +0.3, 3/23+0.57, "A",rotation=0 , transform=ax.transAxes, size=30, **h_font)
ax.text2D(0/13 +0.3, -1.9/23+0.57, "B",rotation=0 , transform=ax.transAxes, size=30, **h_font)

ax.text2D(6/13 +0.3, 2.2/23+0.57, "Branching Patterns",rotation=90 , transform=ax.transAxes, size=30, **h_font)
ax.text2D(6/13 +0.3, -2.2/23+0.57, "Leaf Arrangements",rotation=90 , transform=ax.transAxes, size=30, **h_font)


## 4. Read and write parameter files.
1. CPlantBox use XML file to parameterize the plant, the file could be a little bit scarry at first glance. However, here I will give you examples to try each parameter. In this way you will know all the parameters quickly.
Firstly, we print out all the parameters:

In [ ]:
# check parameters in xml files
name= "PMA2018_new" #name of the small plant
all_parameter = ET.parse("../../modelparameter/plant/{}.xml".format(name)) # read the parameter file from xml file
plant_parameter = all_parameter.getroot() # get the first level of parameters
parameter_options={}
parameter_order = 0
for organ in plant_parameter.iter('organ'): 
    list=[]
    print(organ.attrib ) # print of parameters
    
    for parameter in organ.iter('parameter'): 
        
        print(parameter.attrib)
        
        list.append(parameter.attrib['name']) 
        parameter_order = parameter_order+1
    print("\n" )
    parameter_options[organ.attrib['type']+organ.attrib['subType']] = list



In [ ]:
# here we show what is the plant simulated based on those parameters
name= "PMA2018"
time = 90
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

## 6. Change the length of the Stem
### 6.1 lmax: the maximal length
Length is controled by six parameters, among them, the maximal length is controled by lmax. if we only change the lmax, the growth will only depends on the time. Lets see what will happend if we only change the lmax of the main stem:

In [ ]:

def change_parameter(input_name, output_name, organ_name, subtype , parameter_name, value_type, value):   
    all_parameter = ET.parse("../../modelparameter/{}.xml".format(input_name)) # read the parameter file from xml file
    plant_parameter = all_parameter.getroot() # get the first level of parameters
    original_lmax = plant_parameter.find("./organ[@type='{}'][@subType='{}']/parameter[@name='{}']".format(organ_name, subtype, parameter_name)).get('{}'.format(value_type)) # get function to read the value
    # The '10' in the following line is the value that need to be changed change the value
    plant_parameter.find("./organ[@type='{}'][@subType='{}']/parameter[@name='{}']".format(organ_name, subtype, parameter_name)).set('{}'.format(value_type),'{}'.format(value)) # set function to read the value
    current_lmax = plant_parameter.find("./organ[@type='{}'][@subType='{}']/parameter[@name='{}']".format(organ_name, subtype, parameter_name)).get('{}'.format(value_type)) # get function to read the value
    print('original {} of {} organ with subtype {} is {}, changed to {}'.format(parameter_name, organ_name, subtype ,original_lmax, current_lmax))
    all_parameter.write('../../modelparameter/{}.xml'.format(output_name))
change_parameter('PMA2018', 'testXML', 'stem', '1', 'lmax', 'value','10')
name = 'testXML'
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()
# Try to change the "10" to 5
change_parameter('PMA2018', 'testXML', 'stem', '1', 'lmax', 'value','5')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

### 5.2 lb, ln and la: the parameters controls branching
In the previous section, you may notice that, with the increase of the lmax, the length of the main stem increased. But with the increase length of the stem, we also get more leaves. Is it possible to reduce the leaf numbers? There are two ways, first, we could change the lb, lb is the length at bottom without any branching.

In [ ]:
# change lb to 3 based on the testXML created in last section
name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'lb', 'value','3')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()
# now there is 3 branches again

In [ ]:
# change lb to 3 based on the testXML created in last section
name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'lb', 'value','3')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()
# now there is 3 branches again

### 5.2.2 Let's change the ln to see what will happen?
ln is the internode distance

In [ ]:
name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'value','0.3')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

### 5.2.2Let's change the la to see what will happen?
la is the stem top without branches

In [ ]:
name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'la', 'value','2')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

In [ ]:
# in addition, the functiontype decides what branching internode distance will be
name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','1')
smallplant = CPlantBox(name, time, name) # make a plant object in python
fig = visual_plant(smallplant)    
fig.show()

In [ ]:

fig = make_subplots(
    rows=3, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'lmax', 'value','20')
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','1')
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','0')
smallplant1 = CPlantBox(name, time, name) # make a plant object in python
fig1 = visual_plant_sub(smallplant1,name="functiontype 0")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','1')
smallplant2 = CPlantBox(name, time, name) # make a plant object in python
fig2 = visual_plant_sub(smallplant2,name="functiontype 1")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','2')
smallplant3 = CPlantBox(name, time, name) # make a plant object in python
fig3 = visual_plant_sub(smallplant3,name="functiontype 2")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','3')
smallplant4 = CPlantBox(name, time, name) # make a plant object in python
fig4 = visual_plant_sub(smallplant4,name="functiontype 3")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','2')
smallplant5 = CPlantBox(name, time, name) # make a plant object in python
fig5 = visual_plant_sub(smallplant3,name="functiontype 4")

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'ln', 'functiontype','3')
smallplant6 = CPlantBox(name, time, name) # make a plant object in python
fig6 = visual_plant_sub(smallplant4,name="functiontype 5")

fig.add_trace(fig1, 1, 1 )
fig.add_trace(fig2, 1, 2)
fig.add_trace(fig3, 2, 1)
fig.add_trace(fig4, 2, 2)
fig.add_trace(fig5, 3, 1)
fig.add_trace(fig6, 3, 2)

fig.update_layout(
    title_text='different RotBeta (function type)',
    height=800,
    width=800
)

fig.show()

## 6. Change the rotation of the Stem
### Rotation Theta (rotaBeta) : the angle between main stem and sub branch
### Rotation Beta (rotaBeta) : the angle between two neigbour sub branch
the value of Rotation multiply by Pie is the revlution of the neighbouring sub branch.

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

name = 'testXML'
change_parameter('testXML', 'testXML', 'stem', '1', 'lmax', 'value','20')
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','0')
smallplant1 = CPlantBox(name, time, name) # make a plant object in python
fig1 = visual_plant_sub(smallplant1,name="0 degree rotation") # every two neighbouring branch is 0 degree

name = 'testXML'
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','0.166666') # the rotation angle is equal to 180*0.166666 
smallplant2 = CPlantBox(name, time, name) # make a plant object in python
fig2 = visual_plant_sub(smallplant2,name="30 degree rotation") # every two neighbouring branch is 30 degree

name = 'testXML'
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','0.5') # the rotation angle is equal to 180*0.5 
smallplant3 = CPlantBox(name, time, name) # make a plant object in python
fig3 = visual_plant_sub(smallplant3,name="90 degree rotation") # every two neighbouring branch is 90 degree

name = 'testXML'
change_parameter('testXML', 'testXML', 'leaf', '2', 'RotBeta', 'value','1') # the rotation angle is equal to 180*1 
smallplant4 = CPlantBox(name, time, name) # make a plant object in python
fig4 = visual_plant_sub(smallplant4,name="180 degree rotation") # every two neighbouring branch is 180 degree

fig.add_trace(fig1, 1, 1 )
fig.add_trace(fig2, 1, 2)
fig.add_trace(fig3, 2, 1)
fig.add_trace(fig4, 2, 2)

fig.update_layout(
    title_text='different RotBeta (function type)',
    height=800,
    width=800
)

fig.show()